In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

https://openaccess.uoc.edu/bitstream/10609/148363/3/miravecaTFG0623memoria.pdf

https://www.portalinmobiliario.com/robots.txt


SAAS
https://github.com/Saas-Starter-Kit/Saas-Kit-prisma?tab=readme-ov-file


In [3]:
from webscrapper_portal_inmobiliario import WebScraperPortalInmobiliario

### TODO:
- ~~trackear propiedades en el tiempo para el mismo sector  SE HACE CON LA [LAT LONG]~~
- ~~Sortear baneos de ip por muchas request - sectores con muchas propiedades~~
https://brightdata.com/cp/zones/residential_proxy1/edit 8 dolares el GB --> SOLUCION
- ~~crear tabla intetmedia con 3 tablas y generar id unico...~~
- ~~buscar estandar para nombres de columnas en tablas, en ingles... sin mayusculas~~
- ~~Hacer que se descarguen las cards urls totales [venta + arriendo]~~
- ~~Los polygonos se agregan a una tabla en la base de datos... con una funcion~~
- ~~para correr el programa se ejecuta otra funcion que leera los polygoinos de la base de datos~~
- Que funcione descargando para multiples polygonos
- ejecutar todos los links en tread
---
- Calculos de estadicticas de rentabilidad
- rankear propiedades basado en esto
- grafico mapa donde se encuentran estas propiedades
- extraer imagenes del TOP 3 Propiedades
- plotear en single plot matplotlib. una especie de reporte automatico.


In [4]:
WSPI = WebScraperPortalInmobiliario()

## MAP HANDLING

CREATE NEW MAP GROUP

In [5]:
# WSPI.insert_new_cluser_map("test",None)

LIST MAPS AVALIABLE

In [6]:
WSPI.list_db_maps()

  mapID             geo_ref_name
0     1  depas_General_arriagada
1     2                     test


LOAD A SELECTED MAP INTO VISUALIZATION

In [7]:
WSPI.vis_map(selected_mapID=1)

depas_General_arriagada --> Sucess!


Map(center=[-33.558483599999995, -70.5883168], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

In [8]:
WSPI.start_download(selected_mapID=1, tipo_inmueble="departamento")

proxy enabled!, random session id:  0.7392079604767846


  0%|          | 0/100 [00:00<?, ?it/s]

#### benchamarks

1145 PROPIEDADES ESCRAPIADAS EN 56 MINUTOS -  SINGLE THREAD

# DATA MANAGE
------------------------------------------

In [156]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
from database import *

In [158]:
conn = create_conect_db("real_state.db")
df = get_joined_data_as_dataframe(conn=conn, threshold_date="2024-07-24 15:00")
df = df.T.drop_duplicates().T
df

NameError: name 'create_conect_db' is not defined

# DATA PROCESING
------------------------------------------

In [ ]:
# df_arriendo = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_arriendo_departamento_1143_18_07_24.xlsx")

df_arriendo = df.query("tipo_operacion=='arriendo'")
len(df_arriendo)

In [ ]:
df_arriendo.head()

### precio arriendo segun tipologia

In [ ]:
df_arriendo.groupby(by="n_dormitorios").mean()

In [ ]:
def remove_outliers(self, column_name):
    """ removedor de outliers para columna especificada"""
    Q1 = self[column_name].quantile(0.25)
    Q3 = self[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    condition = (self[column_name] >= lower_bound) & (self[column_name] <= upper_bound)
    return self.loc[condition]

# Attach the function to the pandas DataFrame class
pd.DataFrame.remove_outliers = remove_outliers

In [ ]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
results_precio_arriendo = {}
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:

    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('Price_UF')
    precio_arriendo = int(clean_tipology_df.Price.quantile(0.1))
    results_precio_arriendo[n_dorm] = precio_arriendo
    print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))
    
# df_price_wo_outliers = pd.concat(list_results)
# df_price_wo_outliers

In [ ]:
results_precio_arriendo

### precio por metro cuadrado

In [ ]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
list_results = []
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:
    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('superficie_util')
    list_results.append(clean_tipology_df)

    # print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))

df_price_wo_outliers = pd.concat(list_results)
df_price_wo_outliers["precio_m2"] = df_price_wo_outliers.Price_UF/df_price_wo_outliers.superficie_util
promedio_zona = int(np.mean(df_price_wo_outliers.Price/df_price_wo_outliers.superficie_util))
promedio_zona_UF = np.round(np.mean(df_price_wo_outliers.Price_UF/df_price_wo_outliers.superficie_util),2)

print(f"precio/m2 de la zona es {promedio_zona} CLP , {promedio_zona_UF} UF")

In [ ]:
df_price_wo_outliers

#### CAPRATE BRUTO

In [ ]:
# df_venta = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_venta_departamento_2203_16_07_24.xlsx")

df_venta = df.query("tipo_operacion=='venta'")
len(df_venta)

In [ ]:
df_venta.head()

In [ ]:
df_venta["caprate_bruto"]=np.NaN
for n_dorm in pd.Series(df_venta["n_dormitorios"].unique()).sort_values().values:
    if n_dorm in results_precio_arriendo.keys():
        df_venta.loc[df_venta["n_dormitorios"]==n_dorm, "caprate_bruto" ] = results_precio_arriendo[n_dorm] * 12 / df_venta.query(f"n_dormitorios=={n_dorm}").Price

df_venta.caprate_bruto = df_venta.caprate_bruto.astype(float)

In [ ]:
df_venta

In [ ]:
df_venta.query("caprate_bruto >= .06 and Price < 180000000")

In [ ]:
df_arriendo.query("n_dormitorios==3")

# APRENDIZAJES


webdriver
2. es bueno reiniciar y cerrar el driver
2. el estado de la pagina (restricciones) cambiara solo con el refresh!!
3. Si se aplica un init y luego get/request no retornara NADA (PUEDE QUE SI PERO EN GENERAL NO PORQUE ES UN USER NUEVO SE ASOCIARA CON WEBSCRAPPING)
4. el orden correcto sera

close--> init--> request (primera vez ,  alta probabilida de falla) -->  refresh (recarga y se puede obtener la tabla y ubicaciion)